In [38]:
from hatchet import *
import json
from state import State
from preprocess import PreProcess
import numpy as np
import random

In [39]:
dirname = '/Users/jarus/ucd/Research/Visualisation/projects/CallFlow/.callflow/calc-pi-all'
dataset = 'calc-pi'

In [40]:
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:,.2f}'.format

# Read entire dataset

In [41]:
def replace_str_with_Node(df, graph):
        mapper = {}
        def dfs_recurse(root):
            for node in root.children: 
                mapper[node.callpath[-1]] = Node(node.nid, node.callpath, None)
                dfs_recurse(node)
        for root in graph.roots:
            mapper[root.callpath[-1]] = Node(root.nid, root.callpath, None)
            dfs_recurse(root)
        df['node'] = df['node'].apply(lambda node: mapper[node] if node in mapper else '')
        return df

In [42]:
def write_gf(state, state_name, format_of_df):
    graph_literal = state.graph.to_literal(graph=state.graph, dataframe=state.df)
    print(graph_literal)
    graph_filepath = dirname + '/' + state_name + '/' + format_of_df + '_graph.json'
    with open(graph_filepath, 'w') as graphFile:
        json.dump(graph_literal, graphFile)

    # dump the filtered dataframe to csv.
    df_filepath = dirname + '/' + state_name + '/' + format_of_df + '_df.csv'
    state.df.to_csv(df_filepath)
    
    entire_df_filepath = dirname + '/' + state_name + '/' + 'entire' +'_df.csv'
    state.entire_df.to_csv(entire_df_filepath)

In [43]:
def read_gf(name):
    state = State()
    df_filepath = dirname + '/' + name +  '/filter_df.csv'
    entire_df_filepath = dirname + '/' + name + '/entire_df.csv'
    graph_filepath = dirname + '/' + name + '/filter_graph.json'
    entire_graph_filepath = dirname + '/' + name + '/entire_graph.json'   

    with open(graph_filepath, 'r') as graphFile:
        data = json.load(graphFile)

    state.gf = GraphFrame()
    state.gf.from_literal(data)

    with open(entire_graph_filepath, 'r') as entire_graphFile:
        entire_data = json.load(entire_graphFile)
            
    state.entire_gf = GraphFrame()
    state.entire_gf.from_literal(entire_data)

    state.df = pd.read_csv(df_filepath)
    state.entire_df = pd.read_csv(entire_df_filepath)

    state.graph = state.gf.graph
    state.entire_graph = state.entire_gf.graph

    # replace df['node'] from str to the Node object.
    state.df = replace_str_with_Node(state.df, state.graph)
    state.entire_df = replace_str_with_Node(state.entire_df, state.entire_graph)

    return state

In [44]:
state = read_gf(dataset)

In [46]:
print(state.df)

                   node  rank   time (inc)       time  nid  rank.1  \
0        <program root>     0   999,238.00       0.00    2       0   
1        <program root>     1   999,390.00       0.00    2       1   
2        <program root>     2 1,000,306.00       0.00    2       2   
3        <program root>     3   999,308.00       0.00    2       3   
4                  main     0   999,238.00       0.00    4       0   
..                  ...   ...          ...        ...  ...     ...   
63  <unknown procedure>     3   173,876.00       0.00   54       3   
64     <unknown file>:0     0   197,718.00 197,718.00   55       0   
65     <unknown file>:0     1   244,939.00 244,939.00   55       1   
66     <unknown file>:0     2   221,846.00 221,846.00   55       2   
67     <unknown file>:0     3   173,876.00 173,876.00   55       3   

                   name type            file  line                     module  \
0        <program root>   PF  <unknown file>     0        libmonitor.so.0.0.0 

# Half-calc-pi dataset

In [47]:
random = random.randint(0,100)*0.01
state.df['time (inc)'] = state.df['time (inc)']*random
state.df['time'] = state.df['time']*random
state.entire_df['time (inc)'] = state.entire_df['time (inc)']*random
state.entire_df['time'] = state.entire_df['time']*random

In [48]:
print(random)

0.8


In [49]:
print(state.df)

                   node  rank  time (inc)       time  nid  rank.1  \
0        <program root>     0  799,390.40       0.00    2       0   
1        <program root>     1  799,512.00       0.00    2       1   
2        <program root>     2  800,244.80       0.00    2       2   
3        <program root>     3  799,446.40       0.00    2       3   
4                  main     0  799,390.40       0.00    4       0   
..                  ...   ...         ...        ...  ...     ...   
63  <unknown procedure>     3  139,100.80       0.00   54       3   
64     <unknown file>:0     0  158,174.40 158,174.40   55       0   
65     <unknown file>:0     1  195,951.20 195,951.20   55       1   
66     <unknown file>:0     2  177,476.80 177,476.80   55       2   
67     <unknown file>:0     3  139,100.80 139,100.80   55       3   

                   name type            file  line                     module  \
0        <program root>   PF  <unknown file>     0        libmonitor.so.0.0.0   
1        

In [50]:
i = '0'
write_gf(state, 'calc-pi-random-'+i, 'entire')

DFS on the graph
Root = <program root> [0]
Node = main[1] time (inc) = nan time = nan
- Node = 62:MPI_Finalize[2] time (inc) = nan time = nan
- - Node = PMPI_Finalize[3] time (inc) = nan time = nan
- - - Node = 294:MPID_Finalize[4] time (inc) = nan time = nan
- - - - Node = 162:MPIDI_CH3_Finalize[5] time (inc) = nan time = nan
- - - - - Node = 230:psm_dofinalize[6] time (inc) = nan time = nan
- - - - - - Node = 36:<unknown procedure>[7] time (inc) = nan time = nan
- - - - - - - Node = <unknown procedure>[8] time (inc) = nan time = nan
- - - - - - - - Node = <unknown procedure>[9] time (inc) = nan time = nan
- - - - - - - - - Node = <unknown procedure>[10] time (inc) = nan time = nan
- - - - - - - - - - Node = <unknown file>:0[11] time (inc) = nan time = nan
- - - - - - - - - - Node = <unknown procedure>[12] time (inc) = nan time = nan
- - - - - - - - - - - Node = <unknown procedure>[13] time (inc) = nan time = nan
- - - - - - - - - - - - Node = <unknown file>:0[14] time (inc) = nan tim

IndexError: index 0 is out of bounds for axis 0 with size 0

# 